In [ ]:
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import Checkbox, Box, Dropdown, fixed, interact, interactive, interact_manual, Label, Layout, Textarea

# Override ipywidgets styles
display(HTML('''
<style>
textarea {min-height: 110px !important;}
/* Allow extra-long labels */
.widget-label {min-width: 20ex !important; font-weight: bold;}
.widget-checkbox {padding-right: 300px !important;}
/* Classes for toggling widget visibility */
.hidden {display: none;}
.visible {display: flex;}
</style>
'''))


class ConfigForm(object):
    # Define widgets
    path_caption_msg = '''
    <h4>The path should be entire path after the <code>collection</code> name, including node type, 
    sub-branches, and <code>_id</code>.<br>For example, <code>,Metadata,sub-branch,node_id,</code>.</h4>
    '''
    path_caption = widgets.HTML(value=path_caption_msg)
    caption = widgets.HTML(value='<h3>Configure Your Action</h3>')
    button = widgets.Button(description='Submit')
    action = widgets.Dropdown(options=['Insert', 'Delete', 'Display', 'Update'], value='Insert')
    collection = widgets.Text(value='')
    datatype = widgets.RadioButtons(options=['Metadata', 'Outputs', 'Related', 'Generic'], value='Metadata')
    path = widgets.Text(value='')
    
    # Configure widget layout
    flex = Layout(display='flex', flex_flow='row', justify_content='space-between')

    # Assemble widgets in Boxes
    form_items = [
        Box([Label(value='Action:'), action], layout=flex),
        Box([Label(value='Collection:'), collection], layout=flex),
        Box([Label(value='Node Type:'), datatype], layout=flex),
        Box([Label(value='Path:'), path], layout=flex)
    ]
    
    # Initialise the class object
    def __init__(self, object):
        self.path_caption = ConfigForm.path_caption
        self.caption = ConfigForm.caption
        self.button = ConfigForm.button
        self.action = ConfigForm.form_items[0]
        self.collection = ConfigForm.form_items[1]
        self.datatype = ConfigForm.form_items[2]
        self.path = ConfigForm.form_items[3]

        # Helper method
        def show(widgets, all_widgets):
            for widget in all_widgets:
                if widget in widgets:
                    widget.add_class('visible').remove_class('hidden')
                else:
                    widget.remove_class('visible').add_class('hidden')                

        # Modify widget visibility when the form is changed
        def change_visibility(change):
            box = self.form
            collection_widget = box.children[1]
            datatype_widget = box.children[2]
            path_widget = box.children[3]
            all_widgets = [collection_widget, datatype_widget, path_widget]
            generic_widgets = {
                'Insert': [collection_widget, datatype_widget, path_widget],
                'Update': [collection_widget, datatype_widget, path_widget],
                'Display': [collection_widget, datatype_widget, path_widget],
                'Delete': [collection_widget, datatype_widget, path_widget]
            }

            nongeneric_widgets = {
                'Insert': [collection_widget, datatype_widget],
                'Update': [collection_widget, datatype_widget],
                'Display': [collection_widget, datatype_widget],
                'Delete': [collection_widget, datatype_widget]
            }
            if str(ConfigForm.datatype.value) == 'Generic':
                widget_list = generic_widgets
                self.path_caption.remove_class('hidden')
            else:
                widget_list = nongeneric_widgets

            active = str(ConfigForm.action.value)
            if active == 'Insert':
                show(widget_list['Insert'], all_widgets)
            elif active == 'Delete':
                show(widget_list['Delete'], all_widgets)
            elif active == 'Update':
                show(widget_list['Update'], all_widgets)
            else:
                show(widget_list['Display'], all_widgets)
                            

        def split_list(str):
            seq = str.split('\n')
            for key, value in enumerate(seq):
                seq[key] = value.strip()
            return seq
        
        def handle_submit(values):
            # Save the form values in a dict
            self.values = {'action': ConfigForm.action.value}
            self.values['datatype'] = ConfigForm.datatype.value 
            self.values['collection'] = ConfigForm.collection.value.strip()
            if ConfigForm.action.value == 'Insert' or ConfigForm.action.value == 'Update':
                if ConfigForm.datatype.value == 'Generic':
                    self.values['path'] = ConfigForm.rights.value.strip()
                    self.values['path'] = ',' + self.values['path'].strip(',') + ','
            print('Configuration saved. Values will be available is the cells below.')
            print(self.values)
    
        # Initialise widgets in the container Box
        self.form = Box([self.action, self.datatype, self.collection, self.path],
                   layout=Layout(
                    display='flex',
                    flex_flow='column',
                    border='solid 2px',
                    align_items='stretch',
                    width='50%'))

        # Modify the CSS and set up some helper variables
        box = self.form
        box.children[3].add_class('hidden')
        action_field = box.children[0].children[1]
        nodetype_field = box.children[1].children[1]
        
        # Display the form and watch for changes
        display(self.caption)
        display(box)
        display(self.path_caption)
        display(self.button)
        self.path_caption.add_class('hidden')
        nodetype_field.observe(change_visibility)
        action_field.observe(change_visibility)
        self.button.on_click(handle_submit)

# Instantiate the form - values accessible with e.g. config.values['delete_id]
config = ConfigForm(object)